<a href="https://colab.research.google.com/github/sanyang12/NLP_assigment/blob/main/Copy_of_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [6]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [10]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [28]:
def softmax(x):
    ### FILL CODE
    e_x = np.exp(x - x.max())
    sum_ex = np.sum(np.exp(x-x.max()))
    return e_x / sum_ex


Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [73]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
        ### FILL CODE
        loss = 0
        for y,x in data:
          #x= data[1,:][i]
          #y = data[:,0][i]
          y_i = np.zeros((10,1))
          y_i[y] = 1.0
          x = x.reshape(len(x),1)

          pred = softmax(w@x)

          loss += -np.log(pred)

          loss_grad = -(y_i-pred)@x.T 
          w = w-0.098*loss_grad


        return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [74]:
def predict(w, x):
    ## FILL CODE
    y_pred = softmax(w@x)
    return np.argmax(y_pred)

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [75]:
def compute_accuracy(w, valid_data):
    ## FILL CODE

    correct_pred = 0
    for i in range(len(valid_data)):
      x_valid = valid_data[i][1]
      y_valid = valid_data[i][0]
      index_pred = predict(w,x_valid)
      if index_pred == y_valid :
        correct_pred +=1
        acc = correct_pred/len(valid_data)

    return acc*100


In [76]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 90.100

